In [1]:
from selenium import webdriver

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from requests import get

import time

In [2]:
browser = webdriver.Chrome()
url = 'https://bwfbadminton.com/player/76115/loh-kean-yew'
browser.get(url)

time.sleep(3)

#path to click on 'TOURNAMENTS' tab
path = '//*[@id="app"]/div/div/div[2]/div/ul/li[2]/a'
tournament_tab = browser.find_element_by_xpath(path)

#command to click
tournament_tab.click()

#wait for it to load after click
time.sleep(3)

#path to click on drop down arrow bottom to select 'Year'
path = '//*[@id="app"]/div[1]/div/section/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div/i'
year_tab = browser.find_element_by_xpath(path)

#command to click
year_tab.click()

#wait for it to load after click
time.sleep(3)

#scrape
main_source = browser.page_source
main_soup = BeautifulSoup(main_source, 'html.parser')

#wait for it to scrape
time.sleep(3)

#path to click on Year 2022 to close it
path = '//*[@id="list-item-22-0"]/div/div'
year_tab = browser.find_element_by_xpath(path)

#command to click
year_tab.click()

#wait for it to load after click
time.sleep(3)

year_containers = main_soup.find_all('div', class_='v-list-item__title')

##################################################################

html_source = []
no_of_tourney = []

for i in range(len(year_containers)-1):
    
    browser.execute_script("window.scrollTo(0, 0)")
    
    #path to click on drop down arrow bottom to select 'Year'
    path = '//*[@id="app"]/div[1]/div/section/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div/i'
    year_tab = browser.find_element_by_xpath(path)

    #command to click
    year_tab.click()

    #wait for it to load after click
    time.sleep(3)

    #path to click on '2021'
    x = i+1
    
    path = '//*[@id="list-item-22-' + str(x) +'"]/div/div'
    twoone_tab = browser.find_element_by_xpath(path)

    #command to click
    twoone_tab.click()

    #wait for it to load after click
    time.sleep(3)

    #scrape
    overall_source = browser.page_source
    overall_soup = BeautifulSoup(overall_source, 'html.parser')
    page_containers = overall_soup.find_all('i', class_='fas fa-fw fa-chevron-down')
    print(len(page_containers))
    
    no_of_tourney.append(len(page_containers))
    
    #html_source = []

    for i in range(len(page_containers)):
        #path to click on arrow button for 'VIEW MATCH BREAKDOWN'
        x = 4+i
        path = '//*[@id="app"]/div/div/div[' + str(x) + ']/div[1]/div/div/div/div[4]/a/h2/i'
        view_match1 = browser.find_element_by_xpath(path)

        time.sleep(3)

        #command to click
        view_match1.click()

        #wait for it to load after click
        time.sleep(3)

        #scrape
        source = browser.page_source
        html_source.append(source)

        time.sleep(3)
###################################################################

browser.quit()

10
4
17
10
4
8
16
9
8
1


In [5]:
html_soup = [BeautifulSoup(html_source[i], 'html.parser') for i in range(len(html_source))]

In [15]:
#find total number of matches in the tournamnet

player1 = []
player1_set1 = []
player1_set2 = []
player1_set3 = []
player2 = []
player2_set1 = []
player2_set2 = []
player2_set3 = []
rounds = []
match_time = []
tournament_name = []
year = []

count = 0 
index = 0   

for soup in html_soup:
    
    #each soup contains all the tournament profile in that year. if there are 10 tournaments in year 2021, html_soup[0] to ..
    # .. html_soup[9] will contain the same 10 tournaments info
    #
    tourney_containers = soup.find_all('div', class_='profile-tmt-detail')
    year_containers = soup.find_all('div',class_='v-select__selection v-select__selection--comma')
    
    #each soup represents one tournament
    
    match_containers = soup.find_all('li', class_='result-match-single-card')
    
    #print(len(match_containers))
    
    if count == no_of_tourney[index]:    #number of tournament per year
        count = 0 
        index += 1
    
    #each container represent one match in that particular tournament (soup)
    for container in match_containers:

        tournament_name.append((tourney_containers[count].h2.span.a.text))   #count will only increase after inner loops done
        year.append(year_containers[0].text)
        
        #find first player names across all the rounds
        player1_info = container.find('div', class_='team-details-wrap-card')
        player1_name = player1_info.a.text.strip()
        player1.append(player1_name)
        
        #find first player score
        player1_score = player1_info.find_all('div', class_='score')
        scores1 = player1_score[0].find_all('span')
        if len(scores1) == 3:
            
            scores1_1 = scores1[0].text
            player1_set1.append(scores1_1)

            scores1_2 = scores1[1].text
            player1_set2.append(scores1_2)
            
            scores1_3 = scores1[2].text
            player1_set3.append(scores1_3)
            
        elif len(scores1) == 2:
            
            scores1_1 = scores1[0].text
            player1_set1.append(scores1_1)

            scores1_2 = scores1[1].text
            player1_set2.append(scores1_2)
            
            scores1_3 = None
            player1_set3.append(scores1_3)
            
        elif len(scores1) == 1:

            scores1_1 = scores1[0].text
            player1_set1.append(scores1_1)
            
            scores1_2 = None
            player1_set2.append(scores1_2)
            
            scores1_3 = None
            player1_set3.append(scores1_3)
            
        else:
            scores1_1 = None
            player1_set1.append(scores1_1)
            scores1_2 = None
            player1_set2.append(scores1_2)
            scores1_3 = None
            player1_set3.append(scores1_3)

        #find second player names across all the rounds
        player2_info = container.find('div', class_='team-details-wrap-card margin-top')
        if player2_info != None:
            player2_name = player2_info.a.text.strip()
            player2.append(player2_name)
        else:
            player2_name = None
            player2.append(player2_name)

        #find second player score
        if player2_info != None:
            player2_score = player2_info.find_all('div', class_='score')
            scores2 = player2_score[0].find_all('span')
        else:
            socres2 = []
        if len(scores2) == 3:
            
            scores2_1 = scores2[0].text
            player2_set1.append(scores2_1)

            scores2_2 = scores2[1].text
            player2_set2.append(scores2_2)
            
            scores2_3 = scores2[2].text
            player2_set3.append(scores2_3)
            
        elif len(scores2) == 2:
            
            scores2_1 = scores2[0].text
            player2_set1.append(scores2_1)

            scores2_2 = scores2[1].text
            player2_set2.append(scores2_2)
            
            scores2_3 = None
            player2_set3.append(scores2_3)
            
        elif len(scores1) == 1:

            scores2_1 = scores1[0].text
            player2_set1.append(scores2_1)
            
            scores2_2 = None
            player2_set2.append(scores2_2)
            
            scores2_3 = None
            player2_set3.append(scores2_3)
            
        else:
            scores2_1 = None
            player2_set1.append(scores2_1)
            scores2_2 = None
            player2_set2.append(scores2_2)
            scores2_3 = None
            player2_set3.append(scores2_3)
            
        #find all round number
        round_number = container.find('span', class_='round-oop').text.strip()
        rounds.append(round_number)

        #find all match time
        m_time = container.find('div', class_='time').text.strip()
        match_time.append(m_time)
        
    count += 1   #only increase count after all the matches in the tournament

In [16]:
bwf_lky = pd.DataFrame({'year':year,
                        'tournament':tournament_name,
                        'round':rounds,
                        'match_time': match_time,
                        'player1': player1,
                        'player2': player2,
                        'player1_set1': player1_set1,
                        'player2_set1': player2_set1,
                        'player1_set2': player1_set2,
                        'player2_set2': player2_set2,
                        'player1_set3': player1_set3,
                        'player2_set3': player2_set3,})

In [17]:
bwf_lky.sample(10)

,year,tournament,round,match_time,player1,player2,player1_set1,player2_set1,player1_set2,player2_set2,player1_set3,player2_set3
184,2015,Ciputra Hanoi - Yonex Sunrise Vietnam Internat...,R64,1:00,Riyanto SUBAGJA,LOH Kean Yew,21,23,22,20,21,15
49,2019,IDBI Federal Life Insurance Hyderabad Open 2019,QF,0:33,LOH Kean Yew,Subhankar DEY,21,11,21,16,None,None
178,2015,SCG Badminton Asia Junior Championships 2015,,0:39,Seung Hoon LEE,LOH Kean Yew,17,21,17,21,None,None
80,2019,PRINCESS SIRIVANNAVARI Thailand Masters 2019,Qual. R16,0:35,SUN Fei Xiang,LOH Kean Yew,16,21,10,21,None,None
151,2016,Princess Sirivannavari Thailand Masters 2016,R64,0:32,LOH Kean Yew,Sameer VERMA,19,21,13,21,None,None
87,2018,Scottish Open 2018,R64,0:59,LOH Kean Yew,Jason Anthony HO-SHUE,21,18,13,21,14,21
191,2014,Yonex-Sunrise Indonesian Masters 2014,R32,0:30,Riyanto SUBAGJA,LOH Kean Yew,21,19,21,17,None,None
40,2019,Fuzhou China Open 2019,R32,1:05,LEE Zii Jia,LOH Kean Yew,19,21,21,18,21,12
33,2020,DAIHATSU Indonesia Masters 2020,R32,0:43,LOH Kean Yew,Kanta TSUNEYAMA,24,22,21,9,None,None
119,2018,Austrian Open 2018,Qual. R64,0:00,Joran KWEEKEL,LOH Kean Yew,None,None,None,None,None,None


In [14]:
bwf_lky.to_csv(r'C:\Users\Andy\Desktop\Learning\Dataquest\Project_WebScrape\bwf_lky.csv')